# FASE 1: EXTRACCIÓN DATOS

## Documentación de las APIs

- Spotify 
    *   Documentacion Oficial - (https://developer.spotify.com/documentation/web-api)
    *   Libreria de Spotipy - https://spotipy.readthedocs.io/en/2.24.0/ 
- Last.fm (https://www.last.fm/home)

### 0. Cargar las librerías

In [1]:
pip install spotipy

^C
Note: you may need to restart the kernel to use updated packages.


In [2]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [3]:
import pandas as pd
import requests
import re
import json

### 1. Hay que crear la APP en la web Spotify for Developers, vamos a usar el client_id y el client_secret.

In [4]:
CLIENT_ID = 'tu client_id'
CLIENT_SECRET = 'tu client_secret'

In [5]:
CLIENT_ID = 'df8d47eac61b4f398de976bba8b2334a'
CLIENT_SECRET = '318a4a4405e140798ed7e41f41a5eb1c'

In [6]:
# Autenticación
auth_manager = SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
sp = spotipy.Spotify(auth_manager=auth_manager)

In [7]:
# Géneros seleccionados
generos = ['hip-hop']

# Rango de años
anio_inicio = 2000
anio_fin = 2002

In [8]:
query = "genre:rock year:2000-2002" #cogemos los datos de 2000-2002
offset = 0 # El offset sirve para pasar página
total_results = 0
datos = sp.search(q=query, type='track', limit=50, offset=offset)

In [9]:
datos

{'tracks': {'href': 'https://api.spotify.com/v1/search?offset=0&limit=50&query=genre%3Arock%20year%3A2000-2002&type=track',
  'limit': 50,
  'next': 'https://api.spotify.com/v1/search?offset=50&limit=50&query=genre%3Arock%20year%3A2000-2002&type=track',
  'offset': 0,
  'previous': None,
  'total': 100,
  'items': [{'album': {'album_type': 'compilation',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/0LyfQWJT6nXafLPZqxe9Of'},
       'href': 'https://api.spotify.com/v1/artists/0LyfQWJT6nXafLPZqxe9Of',
       'id': '0LyfQWJT6nXafLPZqxe9Of',
       'name': 'Various Artists',
       'type': 'artist',
       'uri': 'spotify:artist:0LyfQWJT6nXafLPZqxe9Of'}],
     'available_markets': ['AR',
      'AU',
      'AT',
      'BE',
      'BO',
      'BR',
      'BG',
      'CA',
      'CL',
      'CO',
      'CR',
      'CY',
      'CZ',
      'DK',
      'DO',
      'DE',
      'EC',
      'EE',
      'SV',
      'FI',
      'FR',
      'GR',
      'GT',
      'HN

In [10]:
type(datos)

dict

In [11]:
datos['tracks'].keys()

dict_keys(['href', 'limit', 'next', 'offset', 'previous', 'total', 'items'])

In [12]:
bd_hiphop = []
for i in range(2000, 2003):
    print(f'Empezando extracción para el año: {i}')
    query = f'genre:hip-hop year:{i}'
    print(query)
    for x in range(0, 300, 50):
        datos = sp.search(q = query, type = 'track', limit = 50, offset = x)
        bd_hiphop.append(datos)
        print(x)

Empezando extracción para el año: 2000
genre:hip-hop year:2000
0
50
100
150
200
250
Empezando extracción para el año: 2001
genre:hip-hop year:2001
0
50
100
150
200
250
Empezando extracción para el año: 2002
genre:hip-hop year:2002
0
50
100
150
200
250


In [13]:
resultados = []
artistas = []
for i in range(0,18):
    for item in bd_hiphop[i]["tracks"]["items"][:50]:
        resultados.append({
           'artista':item['artists'][0]['name'],
           'genero':'hip-hop',
           'tipo':item['type'],
           'nombre':item['name'], #if item['name'] else "Sin Datos" > programación defensiva! (Usar try except!)
           'anio_lanzamiento':item['album']['release_date'][:4], #if item['album']['release_date'] else "Sin Datos"
           'id':item['id']})
        artistas.append(item['artists'][0]['name'])

In [14]:
len(resultados)

900

In [15]:
resultados

[{'artista': 'Sammy Sam',
  'genero': 'hip-hop',
  'tipo': 'track',
  'nombre': "I'm That Type Of Nigga",
  'anio_lanzamiento': '2000',
  'id': '4OWwxqylVwcpCn52SYKvCq'},
 {'artista': 'Sammy Sam',
  'genero': 'hip-hop',
  'tipo': 'track',
  'nombre': 'Roll Wit The Big Boyz',
  'anio_lanzamiento': '2000',
  'id': '2dCpcf4tKz64Ay5cEVDnGR'},
 {'artista': 'Reefa',
  'genero': 'hip-hop',
  'tipo': 'track',
  'nombre': "Funkin' Def",
  'anio_lanzamiento': '2000',
  'id': '6rSeUcM7DACNPt2oU50Zjy'},
 {'artista': 'Apathy',
  'genero': 'hip-hop',
  'tipo': 'track',
  'nombre': 'Chrome Depot Freestyle (feat. Celph Titled) [Acapella]',
  'anio_lanzamiento': '2000',
  'id': '302aZCCoj2DLW5raahKNx3'},
 {'artista': '2 LIVE CREW',
  'genero': 'hip-hop',
  'tipo': 'track',
  'nombre': 'Drop Your Draws',
  'anio_lanzamiento': '2000',
  'id': '6UpsGIHz0qMl7HTGT3NP1D'},
 {'artista': 'Mr. Sche',
  'genero': 'hip-hop',
  'tipo': 'track',
  'nombre': 'Head Up',
  'anio_lanzamiento': '2000',
  'id': '0bp0BZd5

In [16]:
artistas

['Sammy Sam',
 'Sammy Sam',
 'Reefa',
 'Apathy',
 '2 LIVE CREW',
 'Mr. Sche',
 'Mr. Sche',
 'Filosofia de Rua',
 'DarkRoom Familia',
 'Breezy247',
 'Mr. Sche',
 'Mood Ruff',
 'Young Lo - Carlos Warren',
 'Dan-e-o',
 'Free Radicals',
 'Cali Agents',
 'Askari X',
 'Salaam Remi',
 'Mr. Sche',
 'Kid Sensation',
 'Kid Sensation',
 'LUZER',
 'Honey Family',
 'Tystion',
 'Memphis Bleek',
 'Llwybr Llaethog',
 'Mr Del',
 'P Squad',
 'Cali Agents',
 'Дымовая Завеса',
 'Fabolous',
 'Baby D',
 'Kool Savas',
 'D.J. Pinky',
 'Don Pini',
 'Memphis Bleek',
 'Field Mob',
 'Don Pini',
 'Дымовая Завеса',
 'Honey Family',
 'Remy Johnson',
 'Pluside',
 'Zoxea',
 'Kunga 219',
 'DJ Bomberjack',
 'DJ Bomberjack',
 'CAS',
 'Free Radicals',
 'Kool Savas',
 'Don Pini',
 'Mr. Sche',
 '2 LIVE CREW',
 'C-Bo',
 'Free Radicals',
 'Cali Agents',
 'The Roots',
 'Too $hort',
 'Lyrikal',
 'Mr. Sche',
 'Honey Family',
 'Llwybr Llaethog',
 'DJ Bomberjack',
 'Kunga 219',
 '2 LIVE CREW',
 'DJ Bomberjack',
 'Mr. Sche',
 'Filo

In [17]:
artistas = list(set(artistas))

In [18]:
len(artistas)

297

In [21]:
df_hiphop_resultados = pd.DataFrame(resultados)
df_hiphop_resultados.head()

,artista,genero,tipo,nombre,anio_lanzamiento,id
0,Sammy Sam,hip-hop,track,I'm That Type Of Nigga,2000,4OWwxqylVwcpCn52SYKvCq
1,Sammy Sam,hip-hop,track,Roll Wit The Big Boyz,2000,2dCpcf4tKz64Ay5cEVDnGR
2,Reefa,hip-hop,track,Funkin' Def,2000,6rSeUcM7DACNPt2oU50Zjy
3,Apathy,hip-hop,track,Chrome Depot Freestyle (feat. Celph Titled) [A...,2000,302aZCCoj2DLW5raahKNx3
4,2 LIVE CREW,hip-hop,track,Drop Your Draws,2000,6UpsGIHz0qMl7HTGT3NP1D


In [22]:
df_hiphop_resultados.to_csv('hiphop_resultados.csv')

In [24]:
df_hiphop_artistas = pd.DataFrame(artistas)
df_hiphop_artistas.head()

,0
0,Cali Agents
1,Kris Kross
2,Q Bosilini
3,MwanaFA
4,Don Pini


In [25]:
df_hiphop_artistas.to_csv('hiphop_artistas.csv')